In [1]:
import pandas as pd

bank_df_new = pd.read_csv('bank-prep.csv', sep=',')
bank_df_new.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,...,dec,feb,jan,jul,jun,mar,may,nov,oct,sep
0,58,0,2143,1,0,5,261,1,-1,0,...,0,0,0,0,0,0,1,0,0,0
1,36,0,265,1,1,5,348,1,-1,0,...,0,0,0,0,0,0,1,0,0,0
2,25,0,-7,1,0,5,365,1,-1,0,...,0,0,0,0,0,0,1,0,0,0
3,53,0,-3,0,0,5,1666,1,-1,0,...,0,0,0,0,0,0,1,0,0,0
4,24,0,-103,1,1,5,145,1,-1,0,...,0,0,0,0,0,0,1,0,0,0


In [2]:
# imbalanced-learn 패키지 설치
# pip list로 확인

In [6]:
import numpy as np
from imblearn.under_sampling import RandomUnderSampler

X = np.array(bank_df_new.drop('y', axis=1)) # y를 제외하고 설명변수를 만들어 X로 만든다.
Y = np.array(bank_df_new['y']) # y만 설명변수로 Y를 만든다.
print(np.sum(Y==1), np.sum(Y==0))

sampler = RandomUnderSampler(random_state=42) # 샘플링 조건은 랜덤 샘플링으로 설정한다.
X, Y = sampler.fit_resample(X,Y) # X와 Y를 설정한 조건에 맞춰 샘플링한다.
print(np.sum(Y==1), np.sum(Y==0)) # 언더 샘플링(소수로 개수를 맞춤)으로 y가 1인 것과 y가 0인 건수를 820으로 균형맞춘다.

820 6113
820 820


In [9]:
from imblearn.over_sampling import RandomOverSampler

X = np.array(bank_df_new.drop('y', axis=1))
Y = np.array(bank_df_new[['y',]])
print(np.sum(Y==1), np.sum(Y==0))

sampler = RandomOverSampler(random_state=42) # 오버샘플링에서 랜덤샘플링으로 설정한다. 42는 R에서 set.seed와 같다고 보면 된다.
X,Y = sampler.fit_resample(X,Y)
print(np.sum(Y==1), np.sum(Y==0))

820 6113
6113 6113


In [11]:
# 언더샘플링, 오버샘플링처럼 복사하지 않고 보완하는 방법으로는 SMOTE 기법이 있다.

In [18]:
from sklearn.model_selection import KFold # k-fold 교차검증
from sklearn import tree # 결정 트리 알고리즘
from sklearn.metrics import accuracy_score # 성능 계산을 위한 클래스

kf = KFold(n_splits=10, shuffle=True) # K-fold로 교차검증 실시. 데이터 분할 수는 10, 데이터 세트는 재생 가능하도록 설정했다.
scores=[]

for train_id, test_id in kf.split(X): # 훈련 데이터, 테스트 데이터의 조합을 바꿔가며 모델을 작성하고 성능을 확인한다.
    x = X[train_id]
    y = Y[train_id]
    clf = tree.DecisionTreeClassifier() # 분류를 위해 결정 트리 인스턴스 clf를 생성한다.
    clf.fit(x,y) # 결정 트리 모델을 만든다. criterion으로 지니 계수와 정보소득 중 설정할 수 있다.
    pred_y = clf.predict(X[test_id]) # 작성한 모델에 테스트 데이터를 적용하고 결과를 얻는다.
    score = accuracy_score(Y[test_id], pred_y) # 결과, 정답, 오답 수로부터 모델 성능을 계산한다.
    scores.append(score)
    
scores = np.array(scores)
print(scores.mean(), scores.std()) # 모델의 평균 성능과 표준편차를 계산한다.

0.9577943480989705 0.004720355826538062


In [19]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

print(recall_score(Y[test_id], pred_y)) # 재현율 계산
print(precision_score(Y[test_id], pred_y)) # 적합율 계산

1.0
0.9287878787878788


In [20]:
print(clf)

DecisionTreeClassifier()


In [22]:
from sklearn.model_selection import GridSearchCV

params = {
    'criterion' : ['entropy'], # Criterion은 entropy로 고정한다.
    'max_depth' : [2, 4, 6, 8, 10], # 트리의 최대 심도는 2~10으로 설정한다.
    'min_samples_leaf' : [10, 20, 30, 40, 50], # 잎사귀의 최소 데이터 수는 10~50 사이로 설정한다.
}

clf_gs = GridSearchCV(tree.DecisionTreeClassifier(), params, cv=KFold(n_splits=10, shuffle=True), scoring='accuracy')
clf_gs.fit(X,Y) # 그리드 서치를 시행한다.
# 그리드 서치를 위해 tree의 결정트리 알고리즘을 선택하고 두 번째 인자에는 탐색하는 파라미터 세트, 세 번째 인자에는 검증 방법으로 K-fold를,
# 네 번째 인자에는 모델의 성능을 평가할 방법을 입력한다.

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=True),
             estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['entropy'],
                         'max_depth': [2, 4, 6, 8, 10],
                         'min_samples_leaf': [10, 20, 30, 40, 50]},
             scoring='accuracy')

In [23]:
print(clf_gs.best_score_)
print(clf_gs.best_params_) # 성능이 가장 높을 때 파라미터 조합을 본다.

0.8792726158342623
{'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 10}


In [24]:
clf_best = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 10, min_samples_leaf = 20)
clf_best.fit(X,Y)

print(clf_best.feature_importances_) # feature_importances로 모든 변수의 결과에 대한 영향력을 표시한다. 가장 영향력 높은 변수를 본다.

[0.04208994 0.         0.02961642 0.06148108 0.00793889 0.06043362
 0.46208751 0.02773472 0.06490717 0.01956213 0.00823472 0.00408983
 0.         0.00119321 0.00264274 0.0014427  0.         0.00151272
 0.07440701 0.01534894 0.01661489 0.         0.01695639 0.
 0.01862156 0.00141732 0.00965328 0.03417428 0.00913853 0.00870042
 0.        ]
